In [1]:
!pip install -q amplpy ampltools pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.0 MB/s eta 0:00:00


In [2]:
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "6b31af08-ff1f-429f-ad0b-4bb913b68f75"

from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

if cloud_platform_name() is None:
    ampl = AMPL()
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6300.6669 expiring 20231231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
profit_data = pd.read_excel("/content/drive/MyDrive/645/Datasets/Car_Optimization.xlsx",
sheet_name=0, index_col=0)
CarType = list(profit_data.index)
Profit = profit_data.stack().to_dict()

demand_data = pd.read_excel("/content/drive/MyDrive/645/Datasets/Car_Optimization.xlsx",
        sheet_name=1, index_col=0)
CarClass = list(demand_data.index)
MaxDemand = dict(zip(CarClass, demand_data["Demand"]))

supply_data = pd.read_excel("/content/drive/MyDrive/645/Datasets/Car_Optimization.xlsx",
        sheet_name=2, index_col=0)
MaxCars = dict(zip(CarType, supply_data.Supply))

In [5]:
ampl.eval ('''

reset;

set CarType;
set CarClass;


param MaxCars{i in CarType};
param MaxDemand{j in CarClass};
param Profit{i in CarType, j in CarClass};

var x {i in CarType, j in CarClass} >= 0 integer;

maximize profit_objective: sum{i in CarType, j in CarClass} Profit[i,j] * x[i,j];

subject to
Cars_available_constraint {i in CarType}: sum{j in CarClass} x[i,j] <= MaxCars[i];
Cars_demand_constraint {j in CarClass}: sum{i in CarType} x[i,j] <= MaxDemand[j];
min_cars_per_type_constraint{i in CarType, j in CarClass}: x[i,j] >= 5;


''')

In [6]:
ampl.set['CarType'] = CarType
ampl.set["CarClass"] = CarClass

ampl.param["MaxCars"] = MaxCars
ampl.param["MaxDemand"] = MaxDemand
ampl.param["Profit"] = Profit

In [7]:
ampl.setOption('solver', 'cbc')
ampl.solve()

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 10000
0 simplex iterations
 


In [8]:
ampl.eval('''expand;''')

maximize profit_objective:
	12*x['Compact','Basic'] + 23*x['Compact','Premium'] + 
	33*x['Compact','Ultra'] + 18*x['MidSize','Basic'] + 
	27*x['MidSize','Premium'] + 39*x['MidSize','Ultra'] + 
	25*x['SUV','Basic'] + 35*x['SUV','Premium'] + 48*x['SUV','Ultra'];

subject to Cars_available_constraint['Compact']:
	x['Compact','Basic'] + x['Compact','Premium'] + x['Compact','Ultra']
	 <= 140;

subject to Cars_available_constraint['MidSize']:
	x['MidSize','Basic'] + x['MidSize','Premium'] + x['MidSize','Ultra']
	 <= 150;

subject to Cars_available_constraint['SUV']:
	x['SUV','Basic'] + x['SUV','Premium'] + x['SUV','Ultra'] <= 85;

subject to Cars_demand_constraint['Basic']:
	x['Compact','Basic'] + x['MidSize','Basic'] + x['SUV','Basic'] <= 150;

subject to Cars_demand_constraint['Premium']:
	x['Compact','Premium'] + x['MidSize','Premium'] + x['SUV','Premium']
	 <= 120;

subject to Cars_demand_constraint['Ultra']:
	x['Compact','Ultra'] + x['MidSize','Ultra'] + x['SUV','Ultra'] <= 100;

subjec

In [9]:
object = ampl.get_objective('profit_objective')
print("\n")
print("Total Profit is: ", object.get().value(), "\n")
print("Optimal Car Allocation:")
ampl.display('x');



Total Profit is:  10000.0 

Optimal Car Allocation:
x :=
Compact Basic      20
Compact Premium   110
Compact Ultra       5
MidSize Basic     125
MidSize Premium     5
MidSize Ultra      20
SUV     Basic       5
SUV     Premium     5
SUV     Ultra      75
;

